In [1]:
%load_ext autoreload
%autoreload 2

# Import library
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from IPython.display import display
from glob import glob
import cv2
from sklearn.model_selection import train_test_split

# Install openpyxl if needed for reading Excel files
try:
    import openpyxl
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl"])
    import openpyxl

# Import dependency
from src.preprocessing.generate_metadata import discover_wsi
from src.preprocessing.xml_to_mask import get_mask
from src.preprocessing.annotation_utils import resolve_annotation_path
from src.preprocessing.extract_patches import process_slide
from src.preprocessing.load_wsi import load_wsi
from src.train.train_phase1 import train_phase1

In [2]:
# Configuration
BASE_DIR = 'data'
SOURCES = [
    'Yale_HER2_cohort',
    'Yale_trastuzumab_response_cohort',
    'TCGA_BRCA_Filtered'
]
OUTPUT_CSV = 'outputs/index/wsi_index.csv'

In [3]:
import logging

log_dir = 'outputs/preprocessing/logs'
os.makedirs(log_dir, exist_ok=True)
log_path = os.path.join(log_dir, 'preprocessing.log')

# Configure logging to file only (no console output in notebook)
logger = logging.getLogger('preprocessing')
if not logger.handlers:
    handler = logging.FileHandler(log_path)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    # Prevent propagation to avoid duplicate logs
    logger.propagate = False

def log(msg):
    """Log message using Python logging module (proper file handling)."""
    logger.info(msg)

In [4]:
def create_patch_validator(min_std: float = 5.0, min_foreground_ratio: float = 0.02, background_value: int = 245):
    """Return a validator that drops low-contrast or mostly background patches."""
    stats = {'accepted': 0, 'discarded': 0}
    def _validator(patch, meta):
        arr = np.asarray(patch)
        if arr.ndim == 3:
            gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)
        else:
            gray = arr
        if float(gray.std()) < min_std:
            stats['discarded'] += 1
            return False
        foreground_ratio = float(np.mean(gray < background_value))
        if foreground_ratio < min_foreground_ratio:
            stats['discarded'] += 1
            return False
        stats['accepted'] += 1
        return True
    _validator.stats = stats
    return _validator

In [5]:
csv_path = discover_wsi(
    base_dir=BASE_DIR, 
    sources=SOURCES, 
    output_path=OUTPUT_CSV
)

# Load and display the results
df = pd.read_csv(csv_path)
display(df.head(50))

In [6]:
for row in tqdm(df.head(200).itertuples(index=False), total=200, desc='Processing slides'):
    process_slide(row, base_dir=BASE_DIR)

In [7]:
import os
from glob import glob
from sklearn.model_selection import train_test_split

log("=" * 80)
log("Starting train/val CSV generation")
log("=" * 80)

# Load TCGA label mapping from Excel file
tcga_excel_path = 'data/TCGA_BRCA_Filtered/case&annotation_counts_clean.xlsx'
tcga_labels = {}

if os.path.exists(tcga_excel_path):
    tcga_df = pd.read_excel(tcga_excel_path)
    log(f"Loaded TCGA labels from: {tcga_excel_path}")
    log(f"TCGA cases in Excel: {len(tcga_df)}")
    print(f"Loaded TCGA labels from: {tcga_excel_path}")
    print(f"TCGA cases in Excel: {len(tcga_df)}")
    
    # Create mapping from slide ID to label based on Clinical.HER2.status
    for idx, row in tqdm(tcga_df.iterrows(), total=len(tcga_df), desc='Processing TCGA labels'):
        slide_id = str(row['Slide']).strip()
        her2_status = str(row['Clinical.HER2.status']).strip()
        
        # Map Clinical.HER2.status to label
        if her2_status == 'Positive':
            label = 1  # Positive
        elif her2_status == 'Negative':
            label = 0  # Negative
        else:
            msg = f"Warning: Unknown HER2 status '{her2_status}' for {slide_id}"
            log(msg)
            print(msg)
            continue
        
        tcga_labels[slide_id] = label
    
    pos_count = sum(1 for v in tcga_labels.values() if v == 1)
    neg_count = sum(1 for v in tcga_labels.values() if v == 0)
    log(f"TCGA positive cases: {pos_count}")
    log(f"TCGA negative cases: {neg_count}")
    print(f"TCGA positive cases: {pos_count}")
    print(f"TCGA negative cases: {neg_count}")
else:
    msg = f"Warning: TCGA Excel file not found at {tcga_excel_path}"
    log(msg)
    print(msg)

# Collect all patches with labels
log("Collecting patches from all cases...")
patch_data = []
patches_dir = 'outputs/patches'

# Get all case directories first
case_dirs = [d for d in glob(os.path.join(patches_dir, '*')) if os.path.isdir(d)]
log(f"Found {len(case_dirs)} case directories")

skipped_tcga = []
skipped_unknown = []

for case_dir in tqdm(case_dirs, desc='Collecting patches from cases'):
    case_name = os.path.basename(case_dir)
    
    # Determine label based on case type
    if case_name.startswith('TCGA-'):
        # TCGA cases: look up in Excel file
        # Extract TCGA slide ID (before the UUID)
        # Format: TCGA-A1-A0SP-01Z-00-DX1.UUID
        tcga_id = case_name.split('.')[0] if '.' in case_name else case_name
        
        if tcga_id in tcga_labels:
            label = tcga_labels[tcga_id]
        else:
            msg = f"TCGA case {tcga_id} not found in Excel, skipping"
            skipped_tcga.append(tcga_id)
            tqdm.write(f"Warning: {msg}")
            continue
    
    elif case_name.startswith('S') or case_name.startswith('O'):
        # Yale cases starting with S-* or O-* are positive
        # e.g., S16-32975, O09-03495
        label = 1  # Positive
    
    else:
        # Other Yale cases: determine from directory name
        if 'Her2Pos' in case_name or 'Pos' in case_name:
            label = 1  # Positive
        elif 'Her2Neg' in case_name or 'Neg' in case_name:
            label = 0  # Negative
        else:
            msg = f"Cannot determine label for {case_name}, skipping"
            skipped_unknown.append(case_name)
            tqdm.write(f"Warning: {msg}")
            continue
    
    # Get all PNG files in this directory
    patch_files = glob(os.path.join(case_dir, '*.png'))
    
    for patch_file in patch_files:
        patch_data.append({
            'path': patch_file,
            'label': label,
            'case': case_name
        })

# Log skipped cases
if skipped_tcga:
    log(f"Skipped {len(skipped_tcga)} TCGA cases not found in Excel: {', '.join(skipped_tcga[:5])}{'...' if len(skipped_tcga) > 5 else ''}")
if skipped_unknown:
    log(f"Skipped {len(skipped_unknown)} cases with unknown labels: {', '.join(skipped_unknown[:5])}{'...' if len(skipped_unknown) > 5 else ''}")

log(f"Total patches collected: {len(patch_data)}")
print(f"\nTotal patches found: {len(patch_data)}")

# Create DataFrame
patches_df = pd.DataFrame(patch_data)

# Display label distribution
print("\nLabel distribution:")
print(patches_df['label'].value_counts())
neg_patches = (patches_df['label']==0).sum()
pos_patches = (patches_df['label']==1).sum()
print(f"\nNegative (0): {neg_patches}")
print(f"Positive (1): {pos_patches}")
log(f"Label distribution - Negative: {neg_patches}, Positive: {pos_patches}")

# Display cases per label
cases_by_label = patches_df.groupby('label')['case'].nunique()
neg_cases = cases_by_label.get(0, 0)
pos_cases = cases_by_label.get(1, 0)
print(f"\nNumber of cases:")
print(f"Negative cases: {neg_cases}")
print(f"Positive cases: {pos_cases}")
log(f"Number of cases - Negative: {neg_cases}, Positive: {pos_cases}")

# Split by case to avoid data leakage (patches from same slide stay together)
log("Performing train/val split by case...")
cases = patches_df['case'].unique()
train_cases, val_cases = train_test_split(
    cases, 
    test_size=0.2, 
    random_state=42, 
    stratify=[patches_df[patches_df['case']==c]['label'].iloc[0] for c in cases]
)

train_df = patches_df[patches_df['case'].isin(train_cases)][['path', 'label']]
val_df = patches_df[patches_df['case'].isin(val_cases)][['path', 'label']]

log(f"Train patches: {len(train_df)}, Val patches: {len(val_df)}")
log(f"Train cases: {len(train_cases)}, Val cases: {len(val_cases)}")
print(f"\nTrain patches: {len(train_df)}")
print(f"Val patches: {len(val_df)}")
print(f"Train cases: {len(train_cases)}")
print(f"Val cases: {len(val_cases)}")

# Display split balance
train_neg = (train_df['label']==0).sum()
train_pos = (train_df['label']==1).sum()
val_neg = (val_df['label']==0).sum()
val_pos = (val_df['label']==1).sum()

print(f"\nTrain label distribution:")
print(train_df['label'].value_counts())
print(f"\nVal label distribution:")
print(val_df['label'].value_counts())

log(f"Train split - Negative: {train_neg}, Positive: {train_pos}")
log(f"Val split - Negative: {val_neg}, Positive: {val_pos}")

# Save CSV files
train_csv_path = 'outputs/patches_index_train.csv'
val_csv_path = 'outputs/patches_index_val.csv'

train_df.to_csv(train_csv_path, index=False)
val_df.to_csv(val_csv_path, index=False)

log(f"Saved train CSV to: {train_csv_path}")
log(f"Saved val CSV to: {val_csv_path}")
log("Train/val CSV generation completed successfully")
log("=" * 80)

print(f"\nSaved train CSV to: {train_csv_path}")
print(f"Saved val CSV to: {val_csv_path}")

# Display sample
print("\nSample from train set:")
display(train_df.head())

In [8]:
import os
from pathlib import Path

checkpoint_dir = Path('outputs/phase1/models')
checkpoint_last = checkpoint_dir / 'checkpoint_last.pth'

# Check checkpoint status
if checkpoint_last.exists():
    import torch
    try:
        checkpoint = torch.load(checkpoint_last, map_location='cpu')
        print(f"✓ Checkpoint found: {checkpoint_last}")
        print(f"  - Epoch: {checkpoint['epoch']}")
        print(f"  - Best score ({checkpoint.get('best_metrics', {}).get('score_key', 'auc')}): {checkpoint['best_score']:.4f}")
        print(f"  - Training will resume from epoch {checkpoint['epoch'] + 1}")
        
        # Find all periodic checkpoints
        periodic_checkpoints = sorted(checkpoint_dir.glob('checkpoint_epoch_*.pth'))
        if periodic_checkpoints:
            print(f"\n  Periodic checkpoints ({len(periodic_checkpoints)}):")
            for cp in periodic_checkpoints[-3:]:  # Show last 3
                print(f"    - {cp.name}")
    except Exception as e:
        print(f"✗ Checkpoint exists but cannot be loaded: {e}")
        print(f"  Consider deleting corrupted checkpoint: rm {checkpoint_last}")
else:
    print(f"✗ No checkpoint found at {checkpoint_last}")
    print(f"  Training will start from scratch")

# Helper function to clear checkpoints (uncomment to use)
def clear_checkpoints():
    """Remove all checkpoint files to start fresh"""
    checkpoint_files = list(checkpoint_dir.glob('checkpoint_*.pth'))
    for cp in checkpoint_files:
        cp.unlink()
        print(f"Deleted: {cp.name}")
    if checkpoint_files:
        print(f"\nCleared {len(checkpoint_files)} checkpoint(s)")
    else:
        print("No checkpoints to clear")

# Uncomment the line below to clear all checkpoints and start fresh
# clear_checkpoints()

In [9]:
# Set CUDA memory optimization environment variable
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear any existing CUDA cache
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Check if wandb is installed
try:
    import wandb
    print(f"Weights & Biases version: {wandb.__version__}")
except ImportError:
    print("Warning: wandb not installed. Install with: pip install wandb")
    print("Will continue with TensorBoard logging only")

# CSV Detail: path (image path), label (0=negative, 1=positive)
# Memory optimizations for 7.6GB GPU:
# - Input size: 512x512 (high resolution for better feature extraction)
# - Reduced batch_size: 32 -> 8 (4x less memory)
# - Gradient accumulation: 4 steps (simulates batch_size=32)
# - Mixed precision (FP16): ~50% memory reduction
# - Aggressive cache clearing: Every 10 batches
# Note: 512x512 uses more memory than 224x224. Monitor GPU usage. Reduce to 224 if OOM occurs.

CFG = {
    'train_csv': 'outputs/patches_index_train.csv',
    'val_csv': 'outputs/patches_index_val.csv',
    'output_dir': 'outputs/phase1',
    'pretrained': True,
    'input_size': 512,  # Full resolution patches
    'batch_size': 8,    # Reduced from 32 to 8
    'accumulation_steps': 4,  # Gradient accumulation to simulate batch_size=32
    'use_amp': True,    # Mixed precision training (FP16) for memory efficiency
    'num_workers': 2,   # Reduced to limit CPU memory
    'epochs': 10,
    'lr': 1e-4,
    'weight_decay': 1e-5,
    'label_col': 'label',
    'path_col': 'path',
    'save_best_by': 'auc',
    'seed': 42,
    # Experiment tracking
    'use_wandb': True,  # Enable Weights & Biases logging
    'wandb_project': 'her2-classification',
    'wandb_name': f'phase1_resnet50_bs{8}_size{512}_amp',
    # Checkpoint resumption
    'resume': True,  # Automatically resume from last checkpoint if available
}

log("=" * 80)
log("Starting Phase 1 training (ResNet-50)")
log(f"Configuration: {CFG}")
log(f"Memory optimizations enabled:")
log(f"  - Input size: {CFG['input_size']}x{CFG['input_size']}")
log(f"  - Batch size: {CFG['batch_size']} with {CFG.get('accumulation_steps', 1)} accumulation steps")
log(f"  - Effective batch size: {CFG['batch_size'] * CFG.get('accumulation_steps', 1)}")
log(f"  - Mixed precision (FP16): {CFG.get('use_amp', False)}")
log(f"Experiment tracking:")
log(f"  - TensorBoard: Enabled")
log(f"  - Weights & Biases: {CFG.get('use_wandb', False)}")
log(f"Checkpoint resumption: {CFG.get('resume', False)}")
log("=" * 80)

results = train_phase1(CFG)

log(f"Training completed - Best model: {results['best_model_path']}")
log(f"Best metrics: {results['best_metrics']}")
log(f"TensorBoard logs: {results['tb_dir']}")
if results.get('wandb_url'):
    log(f"Weights & Biases URL: {results['wandb_url']}")
log("=" * 80)

print('\n' + '='*80)
print('TRAINING COMPLETED')
print('='*80)
print(f"Best model: {results['best_model_path']}")
print(f"Logs dir: {results['logs_dir']}")
print(f"TensorBoard dir: {results['tb_dir']}")
print('\nTo view TensorBoard:')
print(f"  tensorboard --logdir={results['tb_dir']}")
if results.get('wandb_url'):
    print('\nWeights & Biases Dashboard:')
    print(f"  {results['wandb_url']}")
print('='*80)

In [10]:
# Set CUDA memory optimization environment variable
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear any existing CUDA cache
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Check if wandb is installed
try:
    import wandb
    print(f"Weights & Biases version: {wandb.__version__}")
except ImportError:
    print("Warning: wandb not installed. Install with: pip install wandb")
    print("Will continue with TensorBoard logging only")

# CSV Detail: path (image path), label (0=negative, 1=positive)
# Memory optimizations for 7.6GB GPU:
# - Input size: 512x512 (high resolution for better feature extraction)
# - Reduced batch_size: 32 -> 8 (4x less memory)
# - Gradient accumulation: 4 steps (simulates batch_size=32)
# - Mixed precision (FP16): ~50% memory reduction
# - Aggressive cache clearing: Every 10 batches
# Note: 512x512 uses more memory than 224x224. Monitor GPU usage. Reduce to 224 if OOM occurs.

CFG = {
    'train_csv': 'outputs/patches_index_train.csv',
    'val_csv': 'outputs/patches_index_val.csv',
    'output_dir': 'outputs/phase1',
    'pretrained': True,
    'input_size': 512,  # Full resolution patches
    'batch_size': 8,    # Reduced from 32 to 8
    'accumulation_steps': 4,  # Gradient accumulation to simulate batch_size=32
    'use_amp': True,    # Mixed precision training (FP16) for memory efficiency
    'num_workers': 2,   # Reduced to limit CPU memory
    'epochs': 10,
    'lr': 1e-4,
    'weight_decay': 1e-5,
    'label_col': 'label',
    'path_col': 'path',
    'save_best_by': 'auc',
    'seed': 42,
    # Experiment tracking
    'use_wandb': True,  # Enable Weights & Biases logging
    'wandb_project': 'her2-classification',
    'wandb_name': f'phase1_resnet50_bs{8}_size{512}_amp',
    # Checkpoint resumption
    'resume': True,  # Automatically resume from last checkpoint if available
}

log("=" * 80)
log("Starting Phase 1 training (ResNet-50)")
log(f"Configuration: {CFG}")
log(f"Memory optimizations enabled:")
log(f"  - Input size: {CFG['input_size']}x{CFG['input_size']}")
log(f"  - Batch size: {CFG['batch_size']} with {CFG.get('accumulation_steps', 1)} accumulation steps")
log(f"  - Effective batch size: {CFG['batch_size'] * CFG.get('accumulation_steps', 1)}")
log(f"  - Mixed precision (FP16): {CFG.get('use_amp', False)}")
log(f"Experiment tracking:")
log(f"  - TensorBoard: Enabled")
log(f"  - Weights & Biases: {CFG.get('use_wandb', False)}")
log(f"Checkpoint resumption: {CFG.get('resume', False)}")
log("=" * 80)

results = train_phase1(CFG)

log(f"Training completed - Best model: {results['best_model_path']}")
log(f"Best metrics: {results['best_metrics']}")
log(f"TensorBoard logs: {results['tb_dir']}")
if results.get('wandb_url'):
    log(f"Weights & Biases URL: {results['wandb_url']}")
log("=" * 80)

print('\n' + '='*80)
print('TRAINING COMPLETED')
print('='*80)
print(f"Best model: {results['best_model_path']}")
print(f"Logs dir: {results['logs_dir']}")
print(f"TensorBoard dir: {results['tb_dir']}")
print('\nTo view TensorBoard:')
print(f"  tensorboard --logdir={results['tb_dir']}")
if results.get('wandb_url'):
    print('\nWeights & Biases Dashboard:')
    print(f"  {results['wandb_url']}")
print('='*80)